In [1]:
import pandas as pd

parent_order = pd.read_csv('parent_order.csv')
child_order = pd.read_csv('child_order.csv')
trade = pd.read_csv("trade.csv")
quote = pd.read_csv("quote.csv")

# add time_stamp
child_order['time_stamp'] = pd.to_datetime( child_order['date'] + ' ' + child_order['time'] )
trade['time_stamp'] = pd.to_datetime( trade['date'] + ' ' + trade['time'] )
quote['time_stamp'] = pd.to_datetime( quote['date'] + ' ' + quote['time'] )

# add notional
child_order['notional'] = child_order['price'] * child_order['size']
trade['notional'] = trade['price'] * trade['size']

# for trade & quote
# slice the data by date, symbol, start-time, end-time
def get_data(df, dt, sy, st, et):
  tmp = df[ df.date==dt ]
  tmp = tmp[ tmp.sym==sy ]

  # normalize all trades after 14:57 as close price
  close_time = dt + ' 14:57'
  close_time = pd.to_datetime( close_time )

  # here delay 1 minute, enough
  if et>=close_time:
    small_delay = dt + ' 15:01'
    small_delay = pd.to_datetime( small_delay )
    et = small_delay
  
  # normalize all trades after 14:57 as close price
  if st>=close_time:
    st = close_time

  tmp = tmp[ tmp.time_stamp>=st ]
  tmp = tmp[ tmp.time_stamp<=et ]
  return tmp

# for child_order
# slice the data by parent-id
def get_data02(df, parent_id):
  tmp = df[ df.parentid==parent_id ]
  return tmp


def cost_equation(benchmark, filled_price, side):
  return 10000 * side * (benchmark - filled_price) / benchmark

def print_list(row_list):
  for i in row_list:
    print(i, '\t', end='')
  print()
  return 


In [23]:
import numpy as np
tca_df = pd.DataFrame()
cols = ['Notional(Million)', 'ADV%', 'Trading speed%', 'Spread', 'Open', 'Arrival', 'iVWAP', 'Close', 'PWP5', 'MOO%', 'MOC%', 'Passive%', 'Aggressive%']


N = len(parent_order)
for i in range(N):
  item = parent_order.iloc[i]
  parent_id = item['orderid']
  qty = item['qty']
  sd = item['side']
  dt = item['date']
  sy = item['sym']
  st = item['starttime']
  st = dt + ' ' + st
  st = pd.to_datetime( st )
  et = item['endtime']
  et = dt + ' ' + et
  et = pd.to_datetime( et )

  #
  row_list = []

  # average filled-price for the order
  tmp = get_data(child_order, dt, sy, st, et)
  n1 = sum( tmp['notional'] )
  v1 = sum( tmp['size'] )
  fp = n1/v1

  # open, close time
  open_tm = dt + ' 9:25'
  open_tm = pd.to_datetime( open_tm )
  close_tm = dt + ' 15:00'
  close_tm = pd.to_datetime( close_tm )  

  # notional 
  tmp = get_data(child_order, dt, sy, st, et)
  ntn = sum( tmp['notional']/1000000 ) # million
  ntn

  # ADV%
  tmp = get_data(trade, dt, sy, open_tm, close_tm) # daily size/volume
  v1 = sum( tmp['size'] )
  tmp = get_data(child_order, dt, sy, st, et) # order size/volume
  v2 = sum( tmp['size'] )
  adv = v2/v1 * 100
  adv

  # Trading speed
  tmp = get_data(trade, dt, sy, st, et) # daily size/volume during order execution
  v1 = sum( tmp['size'] )
  tmp = get_data(child_order, dt, sy, st, et) # order size/volume
  v2 = sum( tmp['size'] )
  ts = v2/v1 * 100
  ts

  # Spread in bps
    # add spread
  quote["spread"] = 10000 * (quote["ask"] - quote["bid"]) / 0.5 / (quote["ask"] + quote["bid"])
  tmp = get_data(quote, dt, sy, st, et) # during order execution
  tmp = tmp.dropna()
  spd = sum(tmp['spread'])/len(tmp['spread']) # average
  spd
  
  #
  #print( ntn, adv, ts, spd )
  row_list.append( round(ntn, 2) )
  row_list.append( round(adv, 2) )
  row_list.append( round(ts, 2) )
  row_list.append( round(spd, 2) )


  # open, close
  tmp = get_data(trade, dt, sy, open_tm, close_tm) # open, close price at that day
  op = tmp.iloc[0]['price'] # first one
  cl = tmp.iloc[len(tmp)-1]['price'] # last one
  op, cl # 19.93 19.63

  # arrival
  ar = op
    # add mid quote
  quote['midpx'] = 0.5 * ( quote['bid'] + quote['ask'] )
    # if order arrives before open, then open price is used
    # market mid quote
  if st<=open_tm: 
    ar = op
  else:
    tmp = get_data(quote, dt, sy, open_tm, st)
    ar = tmp.iloc[len(tmp)-1]['midpx'] # the last one before start-time
  ar

  # iVWAP, interval VWAP - from start-time to end-time
  tmp = get_data(trade, dt, sy, st, et) # during order execution
  n1 = sum( tmp['notional']/1000000 ) # 
  v1 = sum( tmp['size']/1000000 ) #
  ivwap = n1/v1 # weighted average
  ivwap

  # PWP5, trading at 5% of speed, that date after start-time
  tmp = get_data(trade, dt, sy, st, close_tm) # all data at that day after start-time
  num = 0.05 # 5%
  start_index = 0
  end_index = 0
  i = 0
  v5 = 0
  while i<len(tmp):
    v5 = v5 + num*tmp.iloc[i]['size']
    if v5>qty: # the accumulated size <= quantity
      break
    i = i + 1
  end_index = i
  tmp = tmp[start_index: end_index]
  n2 = sum( tmp['notional']/1000000 )
  v2 = sum( tmp['size']/1000000 )
  pwp5 = n2/v2 # weighted average
  pwp5

  # 
  op = cost_equation(op, fp, sd)
  ar = cost_equation(ar, fp, sd)
  ivwap = cost_equation(ivwap, fp, sd)
  cl = cost_equation(cl, fp, sd)
  pwp5 = cost_equation(pwp5, fp, sd)

  #
  #print( op, ar, ivwap, cl, pwp5 )
  row_list.append( round(op, 2) )
  row_list.append( round(ar, 2) )
  row_list.append( round(ivwap, 2) )
  row_list.append( round(cl, 2) )
  row_list.append( round(pwp5, 2) )
  

  # MOO% - Market On Open
  tmp = get_data(child_order, dt, sy, open_tm, open_tm)
  vo = sum( tmp['size'] ) # size when trading at open
  moo = vo/qty * 100
  moo

  # MOC% - Market On Close
  tmp = get_data(child_order, dt, sy, close_tm, close_tm)
  vc = sum( tmp['size'] ) # size when trading at close
  moc = vc/qty * 100
  moc

    # merge child_order with quote
  q1 = get_data(quote, dt, sy, st, et) # quote data during order execution
  q1 = q1[ ['time', 'midpx'] ] 
  c1 = get_data(child_order, dt, sy, st, et) # order exexution
    # before 9:30 as as MOO, after 14:57 as MOC
  tm1 = dt + ' 9:30'
  tm1 = pd.to_datetime( tm1 )
  tm2 = dt + ' 14:57'
  tm2 = pd.to_datetime( tm2 )
  c1 = c1[c1.time_stamp >= tm1]
  c1 = c1[c1.time_stamp <= tm2]
  c1 = c1[ ['parentid', 'childid', 'date', 'sym', 'price', 'size', 'time'] ]
  result = q1.set_index('time').join(c1.set_index('time'))
  result = result.dropna()
  result = result.drop_duplicates(keep='first')
  result['passive'] = sd * np.sign( result['midpx'] - result['price'] ) # 1 means passive, -1 aggressive
  result
  

  # Passive%
  tmp = result
  tmp = tmp[ tmp.passive==1.0 ]
  sp = sum( tmp['size'] )
  passive = sp/qty * 100
  passive

  # Aggressive%
  tmp = result
  tmp = tmp[ tmp.passive<=0.0 ]
  sa = sum( tmp['size'] )
  aggressive = sa/qty * 100
  aggressive

  #print( moo, moc, passive, aggressive )
  row_list.append( round(moo, 2) )
  row_list.append( round(moc, 2) )
  row_list.append( round(passive, 2) )
  row_list.append( round(aggressive, 2) )
  
  #
  row_list = pd.Series(row_list, cols)
  tca_df = tca_df.append([row_list],ignore_index=True)  
  #print('---------------------------------------')

# last row - all
row_list = []
ntn_sum = sum( tca_df['Notional(Million)'] )
#print('All \t', ntn_sum, end='\t')
row_list.append(ntn_sum)

i = 1
while i<=12:
  tmp = tca_df.iloc[:, i] * tca_df.iloc[:, 0] / ntn_sum
  tmp = sum(tmp)
  #print( round(tmp, 2), end='\t')
  row_list.append( round(tmp, 2) )
  i = i + 1

row_list = pd.Series(row_list, cols)
tca_df = tca_df.append([row_list], ignore_index=True)

# set index
tca_df.index = ['V001', 'V002', 'V003', 'V004', 'V005', 'All']
tca_df.index.name = 'OrderID'
tca_df

,Notional(Million),ADV%,Trading speed%,Spread,Open,Arrival,iVWAP,Close,PWP5,MOO%,MOC%,Passive%,Aggressive%
OrderID,,,,,,,,,,,,,
V001,15.89,0.90,1.96,8.60,-30.88,-30.88,-2.53,121.47,0.58,3.19,0.00,32.29,64.53
V002,23.39,0.79,1.60,3.98,8.15,25.83,-4.19,-33.89,-13.12,0.00,0.00,32.85,67.15
V003,4.47,0.35,1.09,7.32,-69.38,-69.38,-6.43,-112.44,-25.39,9.94,0.00,48.27,41.79
V004,28.49,1.82,1.82,4.06,3.77,3.77,-1.66,-10.11,4.98,0.00,3.27,43.32,53.42
V005,33.14,2.27,2.27,7.15,-149.28,-149.28,17.86,-117.69,93.14,0.73,3.00,62.38,33.90
All,105.38,1.53,1.90,5.84,-51.72,-47.79,3.58,-33.72,26.74,1.13,1.83,45.54,51.51
